In [ ]:
from src import *
from datasets import load_dataset

dataset = load_dataset("pixparse/pdfa-eng-wds", streaming=True)
small_dataset = dataset["train"].take(2)
for doc in small_dataset:
    doc_name = doc["__key__"]
    pdf_doc = pdf_bytes_to_doc_stream(pdf_bytes=doc['pdf'], doc_name=f'{doc_name}.pdf')
    pdf_to_milvus_with_sentence_transformer(pdf=pdf_doc, embedding_model="sentence-transformers/all-MiniLM-L6-v2", collection_name="pdfa_eng_wds")
    

In [ ]:
from src import *
client = instant_milvus_client(milvus_uri="http://localhost:19530")
embedder = instant_sentece_transformer_embedder("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:

pdf_to_milvus_with_sentence_transformer(pdf="https://arxiv.org/pdf/2408.09869", embedding_model="sentence-transformers/all-MiniLM-L6-v2", collection_name="testing")